Let's train AlexNet from scratch!

In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
import numpy as np

import datasets
import torch

from datasets import load_dataset
from torchvision import transforms
from typing import Tuple

from trainer import Trainer
from utils import get_device, imshow

/home/a75wu/miniconda3/envs/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setup device
device = get_device()

Using CUDA...


### Training configurations

In [3]:
NUM_EPOCHS = 90 # roughly ~90 cycles specified in alexnet paper
BATCH_SIZE = 512 # original paper uses 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0005
LEARNING_RATE = 0.01
NUM_CLASSES = 200 # tiny imagenet

# Dataset

We're going to use hugging face to load our dummy dataset (Tiny Imagenet) for training.

In [4]:
train_data = load_dataset('zh-plus/tiny-imagenet', split='train')
test_data = load_dataset('zh-plus/tiny-imagenet', split='valid')
print(f'train: {len(train_data)}, test: {len(test_data)}')

train: 100000, test: 10000


In [5]:
class TinyImageNetDataset(torch.utils.data.Dataset):

    def __init__(self, hf_dataset: datasets.Dataset, transforms: transforms.Compose = None) -> None:
        self.dataset = hf_dataset
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.dataset)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        sample = self.dataset[idx]

        if self.transforms:
            # some images are apparently greyscale so we need to convert to RGB
            sample["image"] = self.transforms(sample["image"].convert("RGB"))

        image, label = sample["image"], sample["label"]

        return image, label

In [6]:
# data preprocessing

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.Resize(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_set = TinyImageNetDataset(train_data, train_transform)
test_set = TinyImageNetDataset(test_data, test_transform)

# split into 90-10 train and validation dataset
train_set, val_set = torch.utils.data.random_split(train_set, [0.9, 0.1])

# NOTE: apparently adding num_workers is very important for improving training speed lol
# https://stackoverflow.com/questions/53998282/how-does-the-number-of-workers-parameter-in-pytorch-dataloader-actually-work
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [7]:
print(f"train: {len(train_dataloader)}\nvalidation: {len(val_dataloader)}\ntest: {len(test_dataloader)}")

train: 176
validation: 20
test: 20


In [8]:
image, label = next(iter(train_dataloader))
image.shape

torch.Size([512, 3, 227, 227])

# Observe Pretrained AlexNet
Let's see what the pytorch AlexNet looks like

In [9]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)

Using cache found in /home/a75wu/.cache/torch/hub/pytorch_vision_v0.10.0
/home/a75wu/miniconda3/envs/dev/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/a75wu/miniconda3/envs/dev/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Some of the implementation details of the current pytorch model is a different than the original AlexNet paper but this github issue goes into some detail about it:

https://github.com/pytorch/vision/issues/549

NOTE: 
> "we added the avg_pool to make the model support images of different sizes. For images of size 224x224, the avg_pool is a no-op."

**We will try our best to implement the EXACT model mentioned in the paper instead of the PyTorch implementation**

# Our AlexNet Implementation

In [11]:
from alexnet import AlexNet
model = AlexNet()
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2):

In [12]:
n_params = sum(p.numel() for p in model.parameters())
n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters: {n_params}")
print(f"Number of trainable parameters: {n_trainable_params}")

Number of parameters: 62378344
Number of trainable parameters: 62378344


In [ ]:
alexnet_trainer = Trainer(model,
                          batch_size=BATCH_SIZE,
                          learning_rate=LEARNING_RATE,
                          weight_decay=WEIGHT_DECAY,
                          momentum=MOMENTUM,
                          num_epochs=NUM_EPOCHS,
                          device=device)
alexnet_trainer.train(train_dataloader, val_dataloader)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [01:45<00:00,  1.67it/s, loss=5.49]


INFO | [EPOCH 1] LOSS : train=6.4901194247332485 val=5.394183278083801 | ACCURACY : train=0.003987038508057594 val=0.005164292640984058


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [01:45<00:00,  1.66it/s, loss=5.35]


INFO | [EPOCH 2] LOSS : train=5.36592715436762 val=5.342130470275879 | ACCURACY : train=0.004774946719408035 val=0.005664062686264515


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [01:48<00:00,  1.62it/s, loss=5.34]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [01:45<00:00,  1.67it/s, loss=5.33]


INFO | [EPOCH 4] LOSS : train=5.336833815683018 val=5.330399584770203 | ACCURACY : train=0.005044389516115189 val=0.005543428473174572


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [01:45<00:00,  1.66it/s, loss=5.32]


INFO | [EPOCH 5] LOSS : train=5.327997242862528 val=5.3249314546585085 | ACCURACY : train=0.00508389575406909 val=0.007215073797851801


 43%|████████████████████████████████████████████████████████████████████████████████▌                                                                                                            | 75/176 [00:45<00:55,  1.81it/s, loss=5.31]

# Test

## Performance Metrics
Let's implement Top-1 (accuracy) and Top-5 Error Rate Metrics

https://pytorch.org/ignite/generated/ignite.metrics.TopKCategoricalAccuracy.html

In [ ]:
# test top-k accuracy metric
outputs = torch.tensor([
    [0.7, 0.2, 0.05, 0.05],     # 1 is in the top 2
    [0.2, 0.3, 0.4, 0.1],       # 0 is not in the top 2
    [0.4, 0.4, 0.1, 0.1],       # 0 is in the top 2
    [0.7, 0.05, 0.2, 0.05]      # 2 is in the top 2
])
labels = torch.tensor([1, 0, 0, 2])

k = 2
values, indices = torch.topk(outputs, k)
print(f"top-k indices:\n{indices}\n")

print(f"ground truths:\n{labels}")
print(labels.view(-1, 1).expand_as(indices))
topk_correct = indices.eq(labels.view(-1, 1).expand_as(indices))
accuracy = topk_correct.sum().item() / labels.size(0)

accuracy

## Evaluate

In [ ]:
alexnet_trainer.test(test_dataloader)
alexnet_trainer.plot_metrics()